# Supervised Learning with scikit-learn (Ridge Regression)

In [11]:
# Loading libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Loading classes
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate, RandomizedSearchCV
from sklearn import preprocessing
from scipy.stats import uniform

# Ignoring future warnings for readability reasons
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

# Ridge Regression

Ridge regression introduces a penalty for each coefficient. The penalty is introduced by adding the sum of the squared coefficients of the linear regression model to the loss function. The extent of the penalty is defined by the hyperparameter alpha which is multiplied with the sum of the coefficients. It is important to know that while the coefficients of the standard OLS regression are scale invariant, those of Ridge regression aren't.

The advantage of ridge regression over OLS is that to some extent ridge regression reduces the variance of the predictionms at cost at the expense of a slightly increased bias.

In the following example, riddge regression is applied on sales data using expenditure for different media channels. First, I apply ridge regression with alpha = 0.5 and non standardized data. Then, I standardize the data and show that coefficients differ. Finally, I apply randomized search cross to hyptertune the model.

In [12]:
# Loading data
sales_df = pd.read_csv("datasets/advertising_and_sales_clean.csv")

# Preview data
sales_df.head()

,tv,radio,social_media,influencer,sales
0,16000.0,6566.23,2907.98,Mega,54732.76
1,13000.0,9237.76,2409.57,Mega,46677.90
2,41000.0,15886.45,2913.41,Mega,150177.83
3,83000.0,30020.03,6922.30,Mega,298246.34
4,15000.0,8437.41,1406.00,Micro,56594.18


In [13]:
# Splitting data into X matrix and y vector
X = sales_df.drop(["sales", "influencer"], axis = 1).values
y = sales_df["sales"].values

# Splitting data into train and test sample
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Initialize Ridge regression model
ridge = Ridge(alpha = 0.5)

# Train model
ridge.fit(X_train, y_train)

# Prediction
y_pred = ridge.predict(X_test)

# Compute MSE and R2 squared
mse = mean_squared_error(y_test, y_pred)
r2_squared = r2_score(y_test, y_pred)

# Print results
print("Coefficients: {}".format(ridge.coef_))
print("Mean Squared Error (Ridge regression, alpha = 0.5): {}".format(mse))
print("R2 squared (Ridge regression, alpha = 0.5): {}".format(r2_squared))

Coefficients: [ 3.56337681e+00 -2.63637135e-03 -1.40534040e-02]
Mean Squared Error (Ridge regression, alpha = 0.5): 8321632.178943878
R2 squared (Ridge regression, alpha = 0.5): 0.9990105552987842


The preprocessing library of scikit-learn includes the StandardScaler() which standardizes numerical data to zero mean and standard deviation 1.

In [14]:
# Initialize Ridge regression model
ridge_std = Ridge(alpha = 0.5)

# Standardizing data (note that I do not use the pipe here which would be more efficient on purpose)
X_train_scaler = preprocessing.StandardScaler().fit(X_train)
X_test_scaler  = preprocessing.StandardScaler().fit(X_test)
y_train_scaler = preprocessing.StandardScaler().fit(y_train.reshape(-1, 1))
y_test_scaler  = preprocessing.StandardScaler().fit(y_test.reshape(-1, 1))

X_train_std    = X_train_scaler.transform(X_train)
X_test_std     = X_test_scaler.transform(X_test)
y_train_std    = y_train_scaler.transform(y_train.reshape(-1, 1))
y_test_std     = y_test_scaler.transform(y_test.reshape(-1, 1))

# Train model
ridge_std.fit(X_train_std, y_train_std)

# Prediction
y_pred_std = ridge_std.predict(X_test_std)

# Compute MSE and R2 squared
mse = mean_squared_error(y_test_std, y_pred_std)
r2_squared = r2_score(y_pred_std, y_pred_std)

# Print results
print("Coefficients: {}".format(ridge_std.coef_))
print("Mean Squared Error (Ridge regression, alpha = 0.5): {}".format(mse))
print("R2 squared (Ridge regression, alpha = 0.5): {}".format(r2_squared))

Coefficients: [[ 9.99345684e-01  2.13618293e-04 -3.30236246e-04]]
Mean Squared Error (Ridge regression, alpha = 0.5): 0.000988322344493767
R2 squared (Ridge regression, alpha = 0.5): 1.0


As stated above, coefficients differ when using standardized data since these are not scale invariant in ridge regressions. Next, I apply randomized search cross validation to hypertune alpha eventhough the fit is very good since we know that the larger alpha is, the more biased the model is. Therefore, a small alpha should be preferred.

In [15]:
# Defining k folds
kf = KFold(4, random_state=42, shuffle=True)

# Defining grid using randomly distributed values for alpha between 0 and 50
param_grid = {'alpha': np.linspace(0, 50, num=30)}

# Initializing a new model
ridge = Ridge()

# Defining Randomized Search CV
ridge_cv = RandomizedSearchCV(ridge, param_grid, cv = kf, random_state = 42, n_iter = 10)

# Fitting model using standardized train and test data
ridge_cv.fit(X_train_std, y_train_std)

# Results of GridSearchCV
pd.DataFrame(ridge_cv.cv_results_).iloc[1:,5:].sort_values("rank_test_score")

,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,mean_test_score,std_test_score,rank_test_score
9,{'alpha': 0.0},0.998969,0.998910,0.999010,0.999056,0.998986,0.000054,1
4,{'alpha': 13.793103448275861},0.998882,0.998820,0.998892,0.998960,0.998888,0.000050,2
5,{'alpha': 15.517241379310343},0.998860,0.998795,0.998864,0.998935,0.998863,0.000050,3
8,{'alpha': 20.689655172413794},0.998778,0.998704,0.998765,0.998847,0.998773,0.000051,4
1,{'alpha': 25.86206896551724},0.998677,0.998591,0.998643,0.998737,0.998662,0.000053,5
3,{'alpha': 29.310344827586206},0.998599,0.998503,0.998551,0.998653,0.998576,0.000056,6
2,{'alpha': 39.6551724137931},0.998321,0.998189,0.998224,0.998354,0.998272,0.000068,7
7,{'alpha': 41.37931034482759},0.998268,0.998129,0.998163,0.998297,0.998214,0.000070,8
6,{'alpha': 48.275862068965516},0.998043,0.997873,0.997901,0.998055,0.997968,0.000082,10


In [35]:
# Predictions for best model from RandomizedSearchCV
y_pred_std_ridge_cv = ridge_cv.predict(X_test_std)

# Compute MSE and R2 squared
mse = mean_squared_error(y_test_std, y_pred_std_ridge_cv)
r2_squared = r2_score(y_test_std, y_pred_std_ridge_cv)

# Print results
print("Best alpha value: {}".format(ridge_cv.best_estimator_))
print("Coefficients (best ridge model): {}".format(ridge_cv.best_estimator_.coef_))
print("Mean Squared Error (best ridge model): {}".format(mse))
print("R2 squared (best ridge model): {}".format(r2_squared))

Best alpha value: Ridge(alpha=0.0)
Coefficients (best ridge model): [[ 9.99906362e-01 -2.73041674e-04 -3.31013519e-04]]
Mean Squared Error (best ridge model): 0.0009884330608730249
R2 squared (best ridge model): 0.999011566939127


Interestingly, the best model is the model where alpha equals zero, which is the standard OLS regression model. For comparison, the standard linear regression is fitted and compared to the ridge model with alpha = 0.

In [36]:
# Initializing linear model
ols = LinearRegression()

# Model training
ols.fit(X_train_std, y_train_std)

# Computing predictions with test data
y_pred_std_ols = ols.predict(X_test_std)

# Compute MSE and R2 squared
mse = mean_squared_error(y_test_std, y_pred_std_ols)
r2_squared = r2_score(y_test_std, y_pred_std_ols)

# Print results
print("Coefficients: {}".format(ols.coef_))
print("Mean Squared Error: {}".format(mse))
print("R2 squared: {}".format(r2_squared))

Coefficients: [[ 9.99906362e-01 -2.73041674e-04 -3.31013519e-04]]
Mean Squared Error: 0.0009884330608730227
R2 squared: 0.999011566939127
